In [6]:
import boto3
import sagemaker

import pandas as pd
import numpy as np
import os

In [4]:
#Manage interactions with the Amazon SageMaker APIs and any other AWS services needed
session = sagemaker.Session()
#see the region in which we work
region = session.boto_region_name
print("AWS Region : {}".format(region))

role = sagemaker.get_execution_role()
bucket = session.default_bucket()

AWS Region : us-east-1


In [7]:
#Upload the dataset to S3
prefix = "data_script_mode"
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'data/dataset.csv')).upload_file('predictive_maintenance.csv')

## Processing

In [8]:
from sagemaker import image_uris
processing_image = image_uris.retrieve(framework='sklearn',region='us-east-1',version='0.23-1',image_scope='training')
processing_image

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'

In [9]:
mkdir docker

mkdir: cannot create directory ‘docker’: File exists


In [10]:
%%writefile docker/dockerfile

FROM python:3.7-slim-buster

RUN pip3 install pandas scikit-learn
ENV PYTHONUNBUFFERED=TRUE


COPY processing.py .

Overwriting docker/dockerfile


In [11]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!docker build -t $ecr_repository docker

!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

!aws ecr create-repository --repository-name $ecr_repository

!docker tag {ecr_repository + tag} $processing_repository_uri

!docker push $processing_repository_uri

Sending build context to Docker daemon  4.096kB
Step 1/4 : FROM python:3.7-slim-buster
3.7-slim-buster: Pulling from library/python

b88d5577: Pulling fs layer 
16e23423: Pulling fs layer 
c2981082: Pulling fs layer 
68d15a5c: Pulling fs layer 
Digest: sha256:9bd2bfc822a533f99cbe6b1311d5bf0ff136f776ebac9b985407829f17278935[1A
Status: Downloaded newer image for python:3.7-slim-buster
 ---> 099f4583c701
Step 2/4 : RUN pip3 install pandas scikit-learn
 ---> Running in 218c16d6197b
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
from sagemaker.processing import Processor
from sagemaker.processing import ProcessingInput, ProcessingOutput
processor = Processor(
    role = role,
    image_uri = "222978838857.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container",
    instance_count = 1,
    instance_type = "local",
    entrypoint = ["python3", "processing.py"]
    )

bucket_path = 's3://{}'.format(bucket)

processor.run(
    inputs=[ProcessingInput(source=f"{bucket_path}/{prefix}/data/dataset.csv", destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

INFO:sagemaker:Creating processing-job with name sagemaker-processing-container-2023-06-15-07-37-33-654
INFO:sagemaker.local.local_session:Starting processing job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-1ysj7:
    container_name: y6fb33hfwi-algo-1-1ysj7
    entrypoint:
    - python3
    - processing.py
    - --train-test-split-ratio
    - '0.2'
    environment: []
    image: 222978838857.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container
    networks:
      sagemaker-local:
        aliases:
        - algo-1-1ysj7
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmpq65mb59b/algo-1-1ysj7/config:/opt/ml/config
    - /tmp/tmpq65mb59b/algo-1-1ysj7/output:/opt/ml/output
    - /

Creating y6fb33hfwi-algo-1-1ysj7 ... 
Creating y6fb33hfwi-algo-1-1ysj7 ... done
Attaching to y6fb33hfwi-algo-1-1ysj7
y6fb33hfwi-algo-1-1ysj7 | Received arguments Namespace(train_test_split_ratio=0.2)
y6fb33hfwi-algo-1-1ysj7 | Reading input data from /opt/ml/processing/input/dataset.csv
y6fb33hfwi-algo-1-1ysj7 | Splitting data into train and test sets with ratio 0.2
y6fb33hfwi-algo-1-1ysj7 |       Type  ...  Target
y6fb33hfwi-algo-1-1ysj7 | UDI         ...        
y6fb33hfwi-algo-1-1ysj7 | 7390     0  ...       0
y6fb33hfwi-algo-1-1ysj7 | 9276     0  ...       0
y6fb33hfwi-algo-1-1ysj7 | 2996     0  ...       0
y6fb33hfwi-algo-1-1ysj7 | 5317     1  ...       0
y6fb33hfwi-algo-1-1ysj7 | 357      1  ...       0
y6fb33hfwi-algo-1-1ysj7 | ...    ...  ...     ...
y6fb33hfwi-algo-1-1ysj7 | 9226     0  ...       0
y6fb33hfwi-algo-1-1ysj7 | 4860     0  ...       0
y6fb33hfwi-algo-1-1ysj7 | 3265     2  ...       0
y6fb33hfwi-algo-1-1ysj7 | 9846     0  ...       0
y6fb33hfwi-algo-1-1ysj7 | 2733  

In [13]:
preprocessing_job_description = processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]

for output in output_config["Outputs"]:
    if output["OutputName"] == "train_data":
        preprocessed_training_data = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "test_data":
        preprocessed_test_data = output["S3Output"]["S3Uri"]
        
#Observe the processed data 
training_features = pd.read_csv(preprocessed_training_data + "/dataset_train.csv", index_col = "UDI",nrows=10)
print("Training features shape: {}".format(training_features.shape))
training_features.drop(["Target"], axis=1)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Training features shape: (10, 7)


,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
UDI,,,,,,
7390,0,299.8,310.6,1707,32.5,124
9276,0,298.2,308.7,1605,29.4,47
2996,0,300.5,309.8,1550,37.4,148
5317,1,303.6,312.8,1689,33.1,16
357,1,297.6,308.4,1398,48.9,63
50,1,298.9,309.2,1412,44.1,140
7932,0,300.7,311.7,1499,38.4,205
1524,0,298.3,309.0,1519,36.9,41
7553,0,300.3,311.3,1525,40.7,105


## Training

In [ ]:
training_image = image_uris.retrieve(framework='sklearn',region='us-east-1',version='0.20.0',image_scope='training')

from sagemaker.estimator import Estimator

estimator = Estimator(
    role = role,
    instance_count = 1,
    instance_type = "local",
    image_uri = training_image,
    instanc_type = "local",
    entry_point = "train.py"   
)

estimator.fit({"train" : preprocessed_training_data, "test" : preprocessed_test_data})

In [ ]:
training_job_description = estimator.jobs[-1].describe()
training_job_description
model_data_s3_uri = "{}".format(training_job_description["ModelArtifacts"]["S3ModelArtifacts"])
model_data_s3_uri

## Deploy

In [ ]:
from sagemaker.model import Model

inference_image = image_uris.retrieve(framework='sklearn',region='us-east-1',version='0.20.0',image_scope='training')

model = Model(
    image_uri = inference_image,
    model_data = model_data_s3_uri,
    role = role,
    entry_point = "inference.py",
)

In [ ]:
predictor = model.deploy(    
    initial_instance_count = 1,
    instance_type = "local")

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()